In [1]:
%run base.ipynb

In [2]:
sim = simulator()
x_next, sim_x, sim_u, sim_p, dt, f_ext = sim.model('euler')
x_next_reg, sim_x, sim_u, sim_p_reg, dt, f_ext = sim.model_reg('euler')

In [3]:
# lambdify API to (x,u)->(x_next)
Fsim = ca.Function('Vnext',[sim_x, sim_u, sim_p, dt, f_ext],[x_next])

In [4]:
# lambdify API to (x,u)->(x_next) # forward using regressor parameter set
Fsim_reg = ca.Function('Vnext_reg',[sim_x, sim_u, sim_p_reg, dt, f_ext],[x_next_reg])
Fsim_reg_testf = ca.Function('Vnext',[sim_x, sim_u, sim_p, dt, f_ext], [Fsim_reg(sim_x, sim_u, ca.vertcat(beta_sym,v_c), dt, f_ext)])
lumped_params_from_raw_func = ca.Function('lumped_params_from_raw', [sim_p], [ca.vertcat(beta_sym,v_c)])

In [5]:
blue_eul = np.array([0.1, 0.2, 0.3]) # example euler angles
n_0 = np.array([0.0 ,0.0,0.0, 0.0,0.0,0.0]) # example velocity vector
v_0 = np.array([0.1 ,0.2,0.1,0.1,0.1,0.5]) # example velocity vector
a_0 = np.array([0.1 ,0.2,0.9,0.0,1.0,0.0]) # example acceleration vector
f_ext_0 = np.array([0.0 ,0.0,0.0,1.1,0.0,0.0]) # example force vector
u0 = np.array([0.1 ,0.2,0.1,0.1,0.1,0.5])
x0 = ca.vertcat(n_0,v_0)

In [6]:
lumped_params_from_raw_func(blue.sim_params)

DM([17, 24.2, 26.07, 0.23, -0.23, -0.23, 0.23, 0.28, 0.28, 0.28, 112.815, 114.8, 0, 0, 2.2563, -4.03, -6.22, -5.18, -0.07, -0.07, -0.07, -18.18, -21.66, -36.99, -1.55, -1.55, -1.55, 0, 0, 0, 0, 0, 0])

In [7]:
Fsim(x0, u0, blue.sim_params, 0.4, f_ext_0)

DM([0.0394697, 0.0771658, 0.0380188, 0.224162, 0.0510444, 0.194282, 0.12592, 0.179275, 0.0591091, 0.675984, 0.196787, 0.453456])

In [8]:
Fsim_reg_testf(x0, u0, blue.sim_params, 0.4, f_ext_0)

DM([0.0394697, 0.0771658, 0.0380188, 0.224162, 0.0510444, 0.194282, 0.12592, 0.179275, 0.0591091, 0.675984, 0.196787, 0.453456])